In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler  # Feature Scaling


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import optuna
import xgboost as xgb


In [4]:
df=pd.read_csv("https://raw.githubusercontent.com/shahabas9/Hackathon_project/main/Metadata/train.csv")

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
df=df.drop(columns="fare",axis=1)

In [7]:
df['num_of_passengers']=df['num_of_passengers'].replace(0.0,1.0)

In [8]:
df['tip']=df['tip'].round(2)
df['miscellaneous_fees']=df['miscellaneous_fees'].round(2)
df['total_fare']=df['total_fare'].round(2)

In [9]:
num_pipeline=Pipeline(
    steps=[
    ("imputer",SimpleImputer(strategy='median')),
    ("scaler",StandardScaler())
    ]
)

In [10]:
num_col=['trip_duration', 'distance_traveled', 'num_of_passengers', 'tip','miscellaneous_fees', 'surge_applied']

In [11]:
x=df.drop(columns="total_fare",axis=1)

In [12]:
y=df.total_fare

In [13]:
preprocessor=ColumnTransformer([
    ("numerical_pipeline",num_pipeline,num_col)
])

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=42)

In [15]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())


In [16]:
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [18]:
def objective(trail,data=x,target=y):
  train_x,test_x,train_y,test_y=train_test_split(df,target,test_size=0.3,random_state=36)
  param={
      "tree_method":"gpu_hist",
      'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
      'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
      'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
      'n_estimators' :9000,
      'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
      'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
      'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
      }
  Xgb_reg_model=xgb.XGBRegressor(**param)
  Xgb_reg_model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
  pred_xgb=Xgb_reg_model.predict(test_x)
  mse=mean_squared_error(test_y,pred_xgb)
  return mse

In [21]:
find_params=optuna.create_study()
find_params.optimize(objective,n_trials=10)
find_params.best_trial.params


[I 2023-05-31 07:35:14,738] A new study created in memory with name: no-name-cfd769fe-9704-45f3-9e85-da5ab638ef24
<ipython-input-18-de568ff6952f>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-18-de568ff6952f>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


Streaming output truncated to the last 5000 lines.
[4000]	validation_0-rmse:29.90120
[4001]	validation_0-rmse:29.91167
[4002]	validation_0-rmse:29.87918
[4003]	validation_0-rmse:29.60588
[4004]	validation_0-rmse:29.76597
[4005]	validation_0-rmse:29.98132
[4006]	validation_0-rmse:30.05979
[4007]	validation_0-rmse:29.75546
[4008]	validation_0-rmse:29.81280
[4009]	validation_0-rmse:29.87275
[4010]	validation_0-rmse:29.90934
[4011]	validation_0-rmse:29.95742
[4012]	validation_0-rmse:29.89885
[4013]	validation_0-rmse:29.93272
[4014]	validation_0-rmse:29.88141
[4015]	validation_0-rmse:29.77840
[4016]	validation_0-rmse:29.98710
[4017]	validation_0-rmse:29.87029
[4018]	validation_0-rmse:29.89288
[4019]	validation_0-rmse:29.78124
[4020]	validation_0-rmse:29.78683
[4021]	validation_0-rmse:29.73568
[4022]	validation_0-rmse:29.77512
[4023]	validation_0-rmse:29.84621
[4024]	validation_0-rmse:29.76488
[4025]	validation_0-rmse:29.77810
[4026]	validation_0-rmse:29.71119
[4027]	validation_0-rmse:29.784

[I 2023-05-31 07:36:45,059] Trial 0 finished with value: 1024.0109044408134 and parameters: {'lambda': 0.929866666682915, 'alpha': 0.0005015369005870164, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 1, 'max_depth': 9, 'random_state': 30, 'min_child_weight': 71}. Best is trial 0 with value: 1024.0109044408134.


[0]	validation_0-rmse:158.58822
[1]	validation_0-rmse:156.13436
[2]	validation_0-rmse:154.10577


<ipython-input-18-de568ff6952f>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-18-de568ff6952f>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


...

Streaming output truncated to the last 5000 lines.
[4001]	validation_0-rmse:21.15957
[4002]	validation_0-rmse:21.15988
[4003]	validation_0-rmse:21.16018
[4004]	validation_0-rmse:21.15886
[4005]	validation_0-rmse:21.15849
[4006]	validation_0-rmse:21.15882
[4007]	validation_0-rmse:21.15925
[4008]	validation_0-rmse:21.15838
[4009]	validation_0-rmse:21.15766
[4010]	validation_0-rmse:21.15701
[4011]	validation_0-rmse:21.15748
[4012]	validation_0-rmse:21.15718
[4013]	validation_0-rmse:21.15664
[4014]	validation_0-rmse:21.15727
[4015]	validation_0-rmse:21.15642
[4016]	validation_0-rmse:21.15623
[4017]	validation_0-rmse:21.15718
[4018]	validation_0-rmse:21.15781
[4019]	validation_0-rmse:21.15760
[4020]	validation_0-rmse:21.15696
[4021]	validation_0-rmse:21.15682
[4022]	validation_0-rmse:21.15670
[4023]	validation_0-rmse:21.15746
[4024]	validation_0-rmse:21.15705
[4025]	validation_0-rmse:21.15646
[4026]	validation_0-rmse:21.15739
[4027]	validation_0-rmse:21.15730
[4028]	validation_0-rmse:21.156

[I 2023-05-31 07:49:06,647] Trial 8 finished with value: 465.61784899089764 and parameters: {'lambda': 0.007633241426376091, 'alpha': 0.30142659595161553, 'colsample_bytree': 0.7, 'subsample': 1, 'learning_rate': 0.02, 'max_depth': 11, 'random_state': 20, 'min_child_weight': 127}. Best is trial 2 with value: 430.35066682297065.
<ipython-input-18-de568ff6952f>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-18-de568ff6952f>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


Streaming output truncated to the last 5000 lines.
[4000]	validation_0-rmse:25.21744
[4001]	validation_0-rmse:25.21710
[4002]	validation_0-rmse:25.21714
[4003]	validation_0-rmse:25.21706
[4004]	validation_0-rmse:25.21583
[4005]	validation_0-rmse:25.21586
[4006]	validation_0-rmse:25.21567
[4007]	validation_0-rmse:25.21567
[4008]	validation_0-rmse:25.21157
[4009]	validation_0-rmse:25.21154
[4010]	validation_0-rmse:25.21019
[4011]	validation_0-rmse:25.21018
[4012]	validation_0-rmse:25.21019
[4013]	validation_0-rmse:25.21018
[4014]	validation_0-rmse:25.21018
[4015]	validation_0-rmse:25.20593
[4016]	validation_0-rmse:25.20593
[4017]	validation_0-rmse:25.20668
[4018]	validation_0-rmse:25.20668
[4019]	validation_0-rmse:25.20669
[4020]	validation_0-rmse:25.20669
[4021]	validation_0-rmse:25.20672
[4022]	validation_0-rmse:25.20678
[4023]	validation_0-rmse:25.20678
[4024]	validation_0-rmse:25.20679
[4025]	validation_0-rmse:25.20675
[4026]	validation_0-rmse:25.20675
[4027]	validation_0-rmse:25.206

[I 2023-05-31 07:50:00,799] Trial 9 finished with value: 553.0409437048502 and parameters: {'lambda': 0.02503838102185937, 'alpha': 0.14507856318408402, 'colsample_bytree': 0.2, 'subsample': 0.7, 'learning_rate': 0.01, 'max_depth': 4, 'random_state': 20, 'min_child_weight': 116}. Best is trial 2 with value: 430.35066682297065.


{'lambda': 0.0011983503083929934,
 'alpha': 0.6544145473578737,
 'colsample_bytree': 0.5,
 'subsample': 0.7,
 'learning_rate': 0.02,
 'max_depth': 8,
 'random_state': 30,
 'min_child_weight': 147}

In [23]:
best_params=find_params.best_trial.params


In [24]:
model=xgb.XGBRegressor(**best_params)

In [25]:
model.fit(x_train,y_train)

XGBRegressor(alpha=0.6544145473578737, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.0011983503083929934,
             learning_rate=0.02, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=147, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [26]:
y_pred=model.predict(x_test)

In [27]:
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

0.7622527903850596

In [28]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(x_train,y_train)
y_pred2=model2.predict(x_test)
r2_score(y_test,y_pred2)


0.8345159187814302

In [30]:
model.score(x_train,y_train)

0.7572859625454075

In [31]:
estimators = [ ("random",RandomForestRegressor(n_estimators=100,max_depth=10,random_state=42))]

In [35]:
def eval_model(true,predict):
    mse=mean_squared_error(true,predict)
    mae=mean_absolute_error(true,predict)
    rmse=np.sqrt(mse)
    accuracy=r2_score(true,predict)
    return mse,mae,rmse,accuracy

In [34]:
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [37]:
models={
    "stackingRegressor":StackingRegressor(estimators=estimators,final_estimator=xgb.XGBRegressor(**best_params))
    # "SVM_Regression":SVR()

}

model_list=[]
r2_list=[]


for i in range(len(models)):
    model=list(models.values())[i]
    model.fit(x_train,y_train)


    y_pred=model.predict(x_test)
    mse,mae,rmse,accuracy=eval_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score:",accuracy*100)
    print(model.score(x_train,y_train))

    r2_list.append(accuracy)
    
    print('='*35)
    print('\n')



stackingRegressor
Model Training Performance
RMSE: 44.67192324931585
MAE: 19.183177776444726
R2 score: 79.18913899716027
0.8241235623587441


